In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [15]:
data = pd.read_csv('THPTQG.csv')
data.head()

,Year,school,teaching_method,pretest,posttest
0,2020.0,Truong THPT Chuyên Vinh Phuc,Standard,6.2,7.2
1,2019.0,Truong THPT Chuyên Vinh Phuc,Standard,6.6,7.9
2,2018.0,Truong THPT Chuyên Vinh Phuc,Standard,6.4,7.6
3,2017.0,Truong THPT Chuyên Vinh Phuc,Standard,6.1,7.7
4,2016.0,Truong THPT Chuyên Vinh Phuc,Standard,6.4,7.6


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2135 entries, 0 to 2134
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             2133 non-null   float64
 1   school           2133 non-null   object 
 2   teaching_method  2133 non-null   object 
 3   pretest          2133 non-null   float64
 4   posttest         2133 non-null   float64
dtypes: float64(3), object(2)
memory usage: 83.5+ KB


In [16]:
#Lọc nhiễu các giá trị cực đại hoặc cực tiểu

def outliers(data_temp, k=3):
    q1, q3 = np.percentile(data_temp, [25, 75])
    
    iqr = q3 - q1
    lower_bound = q1 - (k * iqr)
    upper_bound = q3 + (k * iqr)
    
    outliers = [index for index,x in enumerate(data_temp) if x < lower_bound or x > upper_bound]
    
    return outliers

In [21]:
#Xóa các ô rỗng
data.dropna(axis=0, how='any', inplace=True)

x = data.drop(['posttest'],axis=1)
y = data['posttest']

#SỐ hóa dữ liệu
Truong = data['school'].unique()
ppday = data['teaching_method'].unique()

cat_features = np.array([i for i in data.columns.tolist() if data[i].dtype == 'object'])
cont_features = np.array([i for i in data.columns.tolist() if data[i].dtype != 'object'])

from sklearn.preprocessing import LabelEncoder
for i in cat_features:
    enc = LabelEncoder()
    x[i] =enc.fit_transform(x[i])


print(x)

        Year  school  teaching_method  pretest
0     2020.0       1                1      6.2
1     2019.0       1                1      6.6
2     2018.0       1                1      6.4
3     2017.0       1                1      6.1
4     2016.0       1                1      6.4
...      ...     ...              ...      ...
2128  1908.0      14                1      3.9
2129  1907.0      14                1      3.8
2130  1906.0      14                1      4.5
2131  1905.0      14                1      4.6
2132  1904.0      14                1      4.1

[2133 rows x 4 columns]


In [22]:
print(y)

0       7.2
1       7.9
2       7.6
3       7.7
4       7.6
       ... 
2128    5.5
2129    4.6
2130    5.1
2131    5.3
2132    4.8
Name: posttest, Length: 2133, dtype: float64


In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pickle
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
model= LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

with open('linear_regression.pkl','wb') as f:
    pickle.dump(model,f)
score = model.score(X_test, y_test)
print('Độ chính xác của mô hình :', score)

Độ chính xác của mô hình : 0.15268732521307626


In [25]:
data_diem= list(zip(x,y))
print(data_diem)

[('Year', 7.2), ('school', 7.9), ('teaching_method', 7.6), ('pretest', 7.7)]


In [26]:
def predict_price(Year, school,teaching_method,pretest):
    with open('linear_regression.pkl', 'rb') as f:
        mmodel = pickle.load(f)
    return model.predict([[Year, school,teaching_method,pretest]])[0]

In [29]:
Year = float(input('Nhap Nam: '))
school = float(input('Nhap ten truong  : '))
teaching_method = float(input('Nhap phuong phap giang day : '))
pretest = float(input('Nhap diem thi thu : '))


posttest = predict_price(Year, school,teaching_method,pretest)

if posttest > 10:
    posttest = 9.5
elif posttest < 0:
    posttest = 0

print(f'Điểm dự đoán là: {posttest} / 10')


Điểm dự đoán là: 7.746457029731836 / 10


c:\Users\hh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"
